In [1]:
# Importa as bibliotecas
import numpy as np
import pandas as pd

from pandas_datareader import data as pdr
!pip install yfinance
import yfinance as yf
yf.pdr_override()

from matplotlib import pyplot as plt
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [5]:
# Busca os preços das ações
## Define as ações
assets = ['DIVO11.SA', 'PETR4.SA', 'VALE3.SA', 'BRFS3.SA']
## Define a data início
start = '2021-01-01'

# Busca os preços ajustados
prices = pdr.get_data_yahoo(assets, start = start)['Close']

[*********************100%***********************]  4 of 4 completed


In [6]:
# Calcula os retornos e retira dados faltantes
returns = prices.pct_change().dropna()

# Define os pesos de cada ativo (EWP)
weights = np.array([0.25, 0.25, 0.25, 0.25])


# Calcula o retorno do portfolio
returns_portfolio = returns.assign(portfolio = returns.dot(weights))

In [7]:
# Calcula o retorno acumulado
acum_index = (1+returns_portfolio).cumprod()

# Retira a data do index e transforma em coluna (para plotar)
acum = acum_index.reset_index(level = 0)

# Escolhe o tamanho do gráfico
plt.figure(figsize = (15, 10))

# Constrói o gráfico
fig = px.line(acum, x = 'Date', y = assets + ['portfolio'])
fig.show()

<Figure size 1080x720 with 0 Axes>

In [8]:
# Calcula o retorno anualizado

N = len(acum_index)
annualized_return = acum_index.iloc[-1,] ** (252 / N) - 1

In [9]:
# Desvio padrão anualizado dos ativos
sd_annualized = returns.std() * np.sqrt(252)

# Matriz de covariância anualizada
cov_matrix = returns.cov() * 252

# Desvio padrão do portfólio
port_sd_annualized = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

In [10]:
# Junta as Series do desvio padrão
annualized_risk = pd.concat([pd.Series({'portfolio' : port_sd_annualized}), sd_annualized])

# Cria um data frame de risco e retorno
annualized_risk_return = pd.DataFrame({'Risco Anualizado' : annualized_risk,
              'Retorno Anualizado' : annualized_return})

In [11]:
# Plota a relação Risco x Retorno
px.scatter(annualized_risk_return,
           x = 'Risco Anualizado',
           y = 'Retorno Anualizado',
           color =  annualized_risk_return.index,
           size = np.array([5, 5, 5, 5, 5])
           )